## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision.models import resnet18
# from torchvision.models import resnet50
from torchvision import transforms

from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(action='ignore') 

C:\Users\OH\Anaconda3\envs\pytorch1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## Hyperparameter Setting

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

True
1
NVIDIA GeForce GTX 1080 Ti


In [4]:
CFG = {
    'IMG_HEIGHT_SIZE':64,
    'IMG_WIDTH_SIZE':224, # 224
    'EPOCHS':200,
    'LEARNING_RATE':2e-3, # 0.001 = 1e-3, 0.01 = 1e-2, 0.002
    'BATCH_SIZE':64,
    'NUM_WORKERS':0, # 본인의 GPU, CPU 환경에 맞게 설정
    'SEED':41
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load & Train/Validation Split

In [6]:
# df = pd.read_csv('./train.csv')
df = pd.read_csv('C:/Users/OH/python/Dacon/Text Recognition/train.csv', encoding = 'utf-8')

In [7]:
# 제공된 학습데이터 중 1글자 샘플들의 단어사전이 학습/테스트 데이터의 모든 글자를 담고 있으므로 학습 데이터로 우선 배치
df['len'] = df['label'].str.len()
train_v1 = df[df['len']==1]

In [8]:
# 제공된 학습데이터 중 2글자 이상의 샘플들에 대해서 단어길이를 고려하여 Train (80%) / Validation (20%) 분할
df = df[df['len']>1]
train_v2, val, _, _ = train_test_split(df, df['len'], test_size=0.2, random_state=CFG['SEED'])

In [9]:
# 학습 데이터로 우선 배치한 1글자 샘플들과 분할된 2글자 이상의 학습 샘플을 concat하여 최종 학습 데이터로 사용
train = pd.concat([train_v1, train_v2])
print(len(train), len(val))

66251 10637


## Get Vocabulary

In [10]:
# 학습 데이터로부터 단어 사전(Vocabulary) 구축
train_gt = [gt for gt in train['label']]
train_gt = "".join(train_gt)
letters = sorted(list(set(list(train_gt))))
print(len(letters))

2349


In [11]:
vocabulary = ["-"] + letters
print(len(vocabulary))
idx2char = {k:v for k,v in enumerate(vocabulary, start=0)}
char2idx = {v:k for k,v in idx2char.items()}

2350


## CustomDataset

In [12]:
import cv2

class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.train_mode = train_mode
        
    def __len__(self):
        return len(self.img_path_list)
    
    def __getitem__(self, index):
        image = Image.open(self.img_path_list[index]).convert('RGB')
        
        if self.train_mode:
            image = self.train_transform(image)
        else:
            image = self.test_transform(image)
            
        if self.label_list is not None:
            text = self.label_list[index]
            return image, text
        else:
            return image
    
    # Image Augmentation
    def train_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
        ])
        return transform_ops(image)
    
    def test_transform(self, image):
        transform_ops = transforms.Compose([
            transforms.Resize((CFG['IMG_HEIGHT_SIZE'],CFG['IMG_WIDTH_SIZE'])),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
            # 데이터 포인트가 동일한 정도의 스케일(중요도)로 반영
        ])
        return transform_ops(image)

In [13]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

val_dataset = CustomDataset(val['img_path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])

In [14]:
image_batch, text_batch = iter(train_loader).next()
print(image_batch.size(), text_batch)

torch.Size([64, 3, 64, 224]) ('신비', '그나마', '씻기다', '쉴', '일반', '셋', '수입되다', '세', '달리다', '뜁', '선원', '추가', '대량', '예매하다', '콩', '쯩', '못', '사모님', '뱀', '호주머니', '젤', '벌', '불', '베개', '수도권', '회복되다', '날씨', '유산', '견해', '체조', '얘', '튀김', '삼계탕', '어려움', '세', '발자국', '터', '함께', '스타', '샛', '참여하다', '븐', '학생증', '신인', '신청', '양복', '추진하다', '집중하다', '구분되다', '기타', '지구', '꽐', '명예', '대륙', '천장', '신', '칡', '창조', '걋', '나무', '향', '여행사', '강', '흄')


## Model Define

In [15]:
class RecognitionModel(nn.Module):
    def __init__(self, num_chars=len(char2idx), rnn_hidden_size=256): # 256
        super(RecognitionModel, self).__init__()
        self.num_chars = num_chars
        self.rnn_hidden_size = rnn_hidden_size
        
        # CNN Backbone = 사전학습된 resnet18 활용
        # https://arxiv.org/abs/1512.03385
        resnet = resnet18(pretrained=True)
        # CNN Feature Extract
        resnet_modules = list(resnet.children())[:-3]
        self.feature_extract = nn.Sequential(
            *resnet_modules,
            nn.Conv2d(256, 256, kernel_size=(3, 6), stride=1, padding=1), # 256 256 kernel_size = (3, 6), 1024 1024 kernel_size = (4, 14), 
            nn.BatchNorm2d(256), # 256, 2048, 2 
            # 학습 과정에서 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화하는것
            nn.ReLU(inplace=True)
            # Sigmoid와 tanh가 갖는 Gradient Vanishing 문제를 해결하기 위한 함수 0보다 작을 때 0, 0보다 크면 그 값
        )

        # input_feature = 1024, output_feature = 256
        self.linear1 = nn.Linear(1024, rnn_hidden_size) # 1024, 6144, 6
        # 선형 회귀 모델, 입력 1024, 출력 256
        
        # RNN
        # rnn 입력 : [sequence, batch_size, input_size]
        self.rnn = nn.RNN(input_size=rnn_hidden_size, 
                            hidden_size=rnn_hidden_size,
                            bidirectional=True, 
                            batch_first=True)
                            
        # RNN은 은닉층의 노드에서 활성화 함수를 통해 나온 결과값을 출력층 방향으로도 보내면서 다시 은닉층 노드의 다음 계산의 입력으로 보내는 특징
        # input_feature = 256*2, output_feature = num_chars=len(char2idx) = 2350                            
        self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        # 선형 회귀 모델, 입력 512, num_chars = 2350
        
        
    def forward(self, x):
        # CNN
        x = self.feature_extract(x) # [batch_size, channels, height, width]
        # 특징 추출
        x = x.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
        # 순서 바꾸기, tensor의 형태를 바꿈
         
        batch_size = x.size(0) # batch_size
        T = x.size(1)
        x = x.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
        x = self.linear1(x)
        
        # RNN
        x, hidden = self.rnn(x)
        
        output = self.linear2(x)
        output = output.permute(1, 0, 2) # [T==10, batch_size, num_classes==num_features]
        
        return output

In [16]:
# class RecognitionModel(nn.Module):
#     def __init__(self, num_chars=len(char2idx), rnn_hidden_size=256):
#         super(RecognitionModel, self).__init__()
#         self.num_chars = num_chars
#         self.rnn_hidden_size = rnn_hidden_size
        
#         # CNN Backbone = 사전학습된 resnet18 활용
#         # https://arxiv.org/abs/1512.03385
#         resnet = resnet50(pretrained=True)
#         # CNN Feature Extract
#         resnet_modules = list(resnet.children())[:-3]
#         self.feature_extract = nn.Sequential(
#             *resnet_modules,
#             nn.Conv2d(256, 256, kernel_size=(3,6), stride=1, padding=1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(inplace=True)
#         )

#         self.linear1 = nn.Linear(1024, rnn_hidden_size)
        
#         # RNN
#         self.rnn = nn.RNN(input_size=rnn_hidden_size, 
#                             hidden_size=rnn_hidden_size,
#                             bidirectional=True, 
#                             batch_first=True)
#         self.linear2 = nn.Linear(self.rnn_hidden_size*2, num_chars)
        
        
#     def forward(self, x):
#         # CNN
#         x = self.feature_extract(x) # [batch_size, channels, height, width]
#         # print("X1")
#         x = x.permute(0, 3, 1, 2) # [batch_size, width, channels, height]
#         # print("X2")
#         batch_size = x.size(0)
#         T = x.size(1)
#         x = x.view(batch_size, T, -1) # [batch_size, T==width, num_features==channels*height]
#         x = self.linear1(x)
#         # print("X3")
        
#         # RNN
#         x, hidden = self.rnn(x)
#         # print("X4")
        
#         output = self.linear2(x)
#         # print("X5")
#         output = output.permute(1, 0, 2) # [T==10, batch_size, num_classes==num_features]
        
#         return output

## Define CTC Loss

In [17]:
criterion = nn.CTCLoss(blank=0) # idx 0 : '-'

In [18]:
def encode_text_batch(text_batch):
    text_batch_targets_lens = [len(text) for text in text_batch]
    text_batch_targets_lens = torch.IntTensor(text_batch_targets_lens)
    
    text_batch_concat = "".join(text_batch)
    text_batch_targets = [char2idx[c] for c in text_batch_concat]
    text_batch_targets = torch.IntTensor(text_batch_targets)
    
    return text_batch_targets, text_batch_targets_lens

In [19]:
def compute_loss(text_batch, text_batch_logits):
    """
    text_batch: list of strings of length equal to batch size
    text_batch_logits: Tensor of size([T, batch_size, num_classes])
    """
    text_batch_logps = F.log_softmax(text_batch_logits, 2) # [T, batch_size, num_classes]  
    text_batch_logps_lens = torch.full(size=(text_batch_logps.size(1),), 
                                       fill_value=text_batch_logps.size(0), 
                                       dtype=torch.int32).to(device) # [batch_size] 

    text_batch_targets, text_batch_targets_lens = encode_text_batch(text_batch)
    loss = criterion(text_batch_logps, text_batch_targets, text_batch_logps_lens, text_batch_targets_lens)

    return loss

## Train

In [20]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    
    best_loss = np.inf
    cnt = 0
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for image_batch, text_batch in tqdm(iter(train_loader)):
            image_batch = image_batch.to(device)
            
            optimizer.zero_grad()
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        _train_loss = np.mean(train_loss)
        
        _val_loss = validation(model, val_loader, device)
        print(f'Epoch : [{epoch}] Train CTC Loss : [{_train_loss:.5f}] Val CTC Loss : [{_val_loss:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_loss)
        
        if best_loss > _val_loss:
            best_loss = _val_loss
            best_model = model
            torch.save(model.state_dict(), 'C:/Users/OH/Desktop/Study/2022-02-01/Modeling/Text Recognition/open/best_model.pth')
            print('Model Saved.')
            cnt = 0
                
        # 학습 실패 -> 쌓는다 -> 카운트
        if best_loss <= _val_loss:
            cnt = cnt + 1
            print('cnt : ' , cnt)
            if cnt > 5:
                break
    
    return best_model

## Validation

In [21]:
def validation(model, val_loader, device):
    model.eval()
    val_loss = []
    with torch.no_grad():
        for image_batch, text_batch in tqdm(iter(val_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            loss = compute_loss(text_batch, text_batch_logits)
            
            val_loss.append(loss.item())
    
    _val_loss = np.mean(val_loss)
    return _val_loss

## Run!!

In [22]:
model = RecognitionModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

# from torchsummary import summary
# summary(resnet50(pretrained=True).to(device), input_size = (3,64,224), device = "cuda")
# summary(model.to(device), input_size = (3,64,224), device = "cuda")

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:09<00:00, 17.86it/s]


Epoch : [1] Train CTC Loss : [5.84649] Val CTC Loss : [3.54457]
Model Saved.
cnt :  1


100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:09<00:00, 17.91it/s]


Epoch : [2] Train CTC Loss : [2.87456] Val CTC Loss : [1.14398]
Model Saved.
cnt :  1


100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:11<00:00, 15.10it/s]


...

Epoch : [51] Train CTC Loss : [0.00006] Val CTC Loss : [0.11260]
Epoch 00051: reducing learning rate of group 0 to 1.5625e-05.
cnt :  4


100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:08<00:00, 18.64it/s]


Epoch : [52] Train CTC Loss : [0.00006] Val CTC Loss : [0.10843]
cnt :  5


100%|████████████████████████████████████████████████████████████████████████████████| 167/167 [00:09<00:00, 18.48it/s]

Epoch : [53] Train CTC Loss : [0.00006] Val CTC Loss : [0.10797]
cnt :  6


## Inference

In [23]:
# test = pd.read_csv('./test.csv')
test = pd.read_csv('C:/Users/OH/python/Dacon/Text Recognition/test.csv')

In [24]:
# img_path_t = 'C:/Users/OH/Desktop/Study/2022-02-01/Modeling/Text Recognition/open/test'
# img_path_t = 'C:/Users/OH/python/Dacon/Text Recognition/open/test'

In [25]:
test_dataset = CustomDataset(test['img_path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [26]:
def decode_predictions(text_batch_logits):
    text_batch_tokens = F.softmax(text_batch_logits, 2).argmax(2) # [T, batch_size]
    text_batch_tokens = text_batch_tokens.numpy().T # [batch_size, T]

    text_batch_tokens_new = []
    for text_tokens in text_batch_tokens:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new

def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for image_batch in tqdm(iter(test_loader)):
            image_batch = image_batch.to(device)
            
            text_batch_logits = model(image_batch)
            
            text_batch_pred = decode_predictions(text_batch_logits.cpu())
            
            preds.extend(text_batch_pred)
    return preds

In [27]:
predictions = inference(infer_model, test_loader, device)

100%|██████████████████████████████████████████████████████████████████████████████| 1159/1159 [01:58<00:00,  9.81it/s]


## Submission

In [28]:
# 샘플 별 추론결과를 독립적으로 후처리
def remove_duplicates(text):
    if len(text) > 1:
        letters = [text[0]] + [letter for idx, letter in enumerate(text[1:], start=1) if text[idx] != text[idx-1]]
    elif len(text) == 1:
        letters = [text[0]]
    else:
        return ""
    return "".join(letters)

def correct_prediction(word):
    parts = word.split("-")
    parts = [remove_duplicates(part) for part in parts]
    corrected_word = "".join(parts)
    return corrected_word

In [29]:
# submit = pd.read_csv('./sample_submission.csv')
submit = pd.read_csv('C:/Users/OH/python/Dacon/Text Recognition/sample_submission.csv')
submit['label'] = predictions
submit['label'] = submit['label'].apply(correct_prediction)

In [30]:
# submit.to_csv('./submission6.csv', index=False, encoding = 'cp949')
submit.to_csv('./submission3.csv', index=False, encoding = 'utf-8')